In [2]:
!pip install sentence-transformers
!pip install flask pyngrok
#install the Pyterrier framework
!pip install python-terrier
# install the nltk modules
!pip install nl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 3.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [3]:
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from flask import Flask, request, render_template_string
from pyngrok import ngrok
import pandas as pd
import numpy as np

import nltk



import re
import requests
from bs4 import BeautifulSoup


nltk.download('wordnet')
from flask import Flask, request, render_template_string

from nltk.corpus import wordnet
from transformers import BertTokenizer, BertModel


import pyterrier as pt
import torch
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
print(stopwords.words('english'))
if not pt.started():
  pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])


from nltk.corpus import wordnet as wn
nltk.download('punkt')
from nltk.stem import *
from nltk.stem.porter import *
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))






from gensim.models import KeyedVectors
from gensim.models import Word2Vec
from collections import Counter
import time


from flask import Flask, request, render_template_string

from sklearn.metrics.pairwise import cosine_similarity



from transformers import BertTokenizer, BertModel






[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

PyTerrier 0.10.1 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
topics = {
    'Dr. Ahmed Zewail': [
        'https://www.britannica.com/biography/Ahmed-Zewail',
        'https://en.wikipedia.org/wiki/Ahmed_Zewail',
        'https://www.nobelprize.org/prizes/chemistry/1999/zewail/facts/'
    ],
    'Zewail City of Science and Innovation': [
        'https://en.wikipedia.org/wiki/Zewail_City_of_Science,_Technology_and_Innovation',
        'https://zewailcity.edu.eg/main/index.php?lang=en',
        'https://www.sciencedirect.com/science/article/pii/S2090123220300934'
    ],
    'Health-related topics': [
        'https://www.who.int/news-room',
        'https://www.nih.gov/news-events/news-releases',
        'https://www.healthline.com/'
    ],
    'Artificial Intelligence': [
        'https://ai.google/discover/generativeai',
        'https://www.ibm.com/topics/artificial-intelligence',
        'https://www.openai.com/research/'
    ]
}
def scrape_website(url, topic):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        elements = soup.find_all(['p', 'h1', 'h2', 'h3', 'li', 'article'])

        data = []
        for idx, element in enumerate(elements):
            text = element.get_text(strip=True)
            if text:
                data.append({
                    'docno': f"{idx}-{element.name}",
                    'Text': text,
                    'Link': url,
                    'Topic': topic
                })
        return data
    except requests.RequestException as e:
        print(f"Failed to retrieve data from {url}: {str(e)}")
        return []
all_data = []
for topic, websites in topics.items():
    for website in websites:
        website_data = scrape_website(website, topic)
        all_data.extend(website_data)

df = pd.DataFrame(all_data)
for topic in df['Topic'].unique():
    topic_rows = df[df['Topic'] == topic]
    print(f"Topic: {topic}, Rows: {topic_rows.index.min()} to {topic_rows.index.max()}")


Failed to retrieve data from https://www.sciencedirect.com/science/article/pii/S2090123220300934: 403 Client Error: Forbidden for url: https://www.sciencedirect.com/science/article/pii/S2090123220300934
Failed to retrieve data from https://www.openai.com/research/: 403 Client Error: Forbidden for url: https://www.openai.com/research/
Topic: Dr. Ahmed Zewail, Rows: 0 to 800
Topic: Zewail City of Science and Innovation, Rows: 801 to 999
Topic: Health-related topics, Rows: 1000 to 1583
Topic: Artificial Intelligence, Rows: 1584 to 1698


In [5]:
def remove_stopwords(text):
  tokens=word_tokenize(text)
  filtered_tokens=[word.lower()for word in tokens if word not in stop_words]
  return' '.join(filtered_tokens)

# def stem_words(text):
#   tokens=word_tokenize(text)
#   stemmed_tokens=[stemmer.stem(word)for word in tokens]
#   return ' '.join(stemmed_tokens)

def clean(text):
    text = re.sub(r"http\S+", " ", text)
    text = re.sub(r"RT ", " ", text)
    text = re.sub(r"@[\w]*", " ", text)
    text = re.sub(r"[.,#_|:?/?=&()]", " ", text)
    text = re.sub(r'\t', ' ', text)
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r"\s+", " ", text)
    text = text.strip()
    return text

def preprocess(text):
    text = remove_stopwords(text)
    # text = stem_words(text)
    text = clean(text)
    return text

In [6]:

df['processed_text'] = df['Text'].apply(preprocess)
df['docno'] = df['docno'].astype(str)

In [7]:
all_terms = []
for text in df['processed_text']:
    tokens = word_tokenize(text)
    all_terms.extend(tokens)


term_counts = Counter(all_terms)

threshold =10
terms = [term for term, count in term_counts.items() if count >= threshold]



In [8]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-mpnet-base-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [40]:

'''interface with photos <<RUN this>>'''
import time
from flask import Flask, request, render_template_string
import pandas as pd
import pyterrier as pt
from sentence_transformers import SentenceTransformer, util
from pyngrok import ngrok
import nltk
from nltk.corpus import wordnet as wn
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

indexer = pt.DFIndexer("./myFirstIndex2", overwrite=True)
index_ref = indexer.index(df["processed_text"], df["docno"], df["Link"], df["Text"])
index = pt.IndexFactory.of(index_ref)
def calculate_precision(retrieved, relevant):

    true_positives = set(retrieved).intersection(relevant)
    if len(retrieved) == 0:
        return 0
    return len(true_positives) / len(retrieved)

def calculate_recall(retrieved, relevant):

    true_positives = set(retrieved).intersection(relevant)
    if len(relevant) == 0:
        return 0
    return len(true_positives) / len(relevant)

def calculate_f1(precision, recall):

    if (precision + recall) == 0:
        return 0
    return 2 * (precision * recall) / (precision + recall)


judgment = {
    'ahmed zewail': list(df.iloc[0:801]['docno']),
    'dr zewail': list(df.iloc[0:801]['docno']),
    'ahmed': list(df.iloc[0:801]['docno']),
    'nobel': list(df.iloc[0:801]['docno']),
    'zewail city for science and technology': list(df.iloc[801:1000]['docno']),
    'zewail city': list(df.iloc[801:1000]['docno']),
    'world health organization': list(df.iloc[1000:1600]['docno']),
    'who': list(df.iloc[1000:1600]['docno']),
    'healthline': list(df.iloc[1000:1600]['docno']),
    'health': list(df.iloc[1000:1600]['docno']),
    'ai': list(df.iloc[1600:1715]['docno']),
    'AI': list(df.iloc[1600:1715]['docno']),
    'artificial intelligence': list(df.iloc[1600:1715]['docno']),

}
app = Flask(__name__)



def get_synonyms(word):
    synonyms = set()
    for syn in wn.synsets(word):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name().replace('_', ' '))
    return synonyms





# def expand_query_with_word2vec(query, terms, word2vec_model_path, top_n=20):

#     word2vec_model = Word2Vec.load(word2vec_model_path)


#     query_embedding = word2vec_model.wv[query]


#     term_embeddings = [word2vec_model.wv[term] for term in terms]


#     cos_scores = [word2vec_model.most_similar([query_embedding], [term_embedding])[0][0] for term_embedding in term_embeddings]


#     related_terms = [terms[i] for i in np.argsort(cos_scores)[::-1][:top_n]]

#     return related_terms
def expand_query_with_embeddings(query, model):
    query_embedding = model.encode(query, convert_to_tensor=True)

    term_embeddings = model.encode(terms, convert_to_tensor=True)
    cos_scores = util.pytorch_cos_sim(query_embedding, term_embeddings)[0]
    related_terms = []
    sorted_indices = cos_scores.argsort(descending=True)
    for i in range(7):
        related_terms.append(terms[sorted_indices[i]])

    return related_terms


@app.route("/", methods=["GET"])
def home():
    return render_template_string(HTML_TEMPLATE)

@app.route("/search", methods=["POST"])
def search():
    start_time = time.time()
    query = request.form['query']
    query = query.lower()
    processed_query = preprocess(query)
    print("Query:", query)
    print("Processed Query:", processed_query)

    synonyms = get_synonyms(processed_query)
    expanded_query = " ".join(synonyms)
    additional_terms = expand_query_with_embeddings(processed_query, model)
    expanded_query += " " + " ".join(additional_terms)

    bm25_retriever = pt.BatchRetrieve(index, wmodel="BM25", num_results=500, metadata=["docno", "Text"])
    final_results = bm25_retriever.transform(expanded_query)

    retrieval_duration = time.time() - start_time
    retrieval_time = f"{retrieval_duration:.2f} seconds"
    result_count = len(final_results)
    retrieval_duration = time.time() - start_time
    retrieval_time = f"{retrieval_duration:.2f} seconds"


    print("Retrieval Time:", retrieval_time)
    print("Result Count:", result_count)
    print("exopanded query is :",expanded_query)

    if 'Text' in final_results.columns:
        if query in judgment:
            retrieved_docnos = set(final_results['docno'].tolist())

            relevant_docnos = set(judgment[query])
            precision = calculate_precision(set(retrieved_docnos), relevant_docnos)
            recall = calculate_recall(set(retrieved_docnos), relevant_docnos)
            f1_score = calculate_f1(precision, recall)
        else:



            precision = 0
            recall = 0
            f1_score = 0

    result_count = len(final_results)
    search_results = []
    for idx, row in final_results.iterrows():
        docno = row['docno']
        link = df[df['docno'] == docno]['Link'].values[0]
        text = df[df['docno'] == docno]['Text'].values[0]
        search_results.append((docno, link, text))

    return render_template_string(HTML_TEMPLATE, query=query, results=search_results, precision=precision, recall=recall, f1_score=f1_score, retrieval_time=retrieval_time, result_count=result_count,expanded_query=expanded_query)

ngrok.set_auth_token('2feTTlkc3MwE8uIJWWxU5M4WRVC_7FhdKJQXYdDY7akug4zJn')
public_url = ngrok.connect(5000)
print('NGROK Tunnel URL:', public_url)


HTML_TEMPLATE = '''
<!DOCTYPE html>
<html lang="en">
<head>
<link rel="stylesheet" href="https://fonts.googleapis.com/css2?family=Raleway:wght@400;700&display=swap">

    <meta charset="UTF-8">
    <title>QUIXY Search Engine</title>
    <style>
        body, html {
            font-family: 'Arial', sans-serif;
            margin: 0;
            padding: 0;
            height: 100%;
            overflow: hidden;
            background: transparent;
        }
        .background {
            position: fixed;
            top: 0;
            left: 0;
            right: 0;
            bottom: 0;
            background: url('https://images.unsplash.com/photo-1487875961445-47a00398c267?crop=entropy&cs=tinysrgb&fit=crop&fm=jpg&h=900&ixid=MnwxfDB8MXxyYW5kb218MHx8dGVjaG5vbG9neXx8fHx8fDE3MTU0NjEzNTQ&ixlib=rb-4.0.3&q=80&utm_campaign=api-credit&utm_medium=referral&utm_source=unsplash_source&w=1600') no-repeat center center fixed;
            background-size: cover;
            filter: blur(4px);
            z-index: -1;
        }
        .content {
            position: relative;
            padding: 30px;
            background: rgba(255, 255, 255, 0.22); /* less transparency for better readability */
            overflow-y: auto;
            height: 100%;
        }
        header {
            color: white;
            text-align: center;
            padding-bottom: 20px;
        }
        form {
            display: flex;
            justify-content: center;
            margin-top: 20px;
        }
        input[type="text"], input[type="submit"] {
            padding: 10px;
            margin: 5px;
            border-radius: 5px;
            border: 1px solid #ccc;
        }
        input[type="text"] {
            flex-grow: 1;
            max-width: 600px;
        }
        input[type="submit"] {
            background-color: #0056b3;
            color: white;
            cursor: pointer;
        }
        input[type="submit"]:hover {
            background-color: #003d7a;
        }
        ul {
            list-style: none;
            padding: 0;
            margin: 0;
        }
        li {
            background-color: #fff;
            padding: 15px;
            margin-top: 10px;
            border-radius: 8px;
            box-shadow: 0 2px 5px rgba(0,0,0,0.1);
            transition: transform 0.3s ease-in-out, box-shadow 0.3s ease-in-out;
        }
        li:hover {
            transform: translateY(-5px);
            box-shadow: 0 4px 12px rgba(0,0,0,0.3);
        }
        a {
            color: #0066cc;
            text-decoration: none;
        }
        a:hover {
            text-decoration: underline;
        }
.statistics {
    text-align: center;
    margin-top: 20px;
    font-family: 'Raleway', sans-serif; /* Apply Raleway font */
    font-weight: bold; /* Make it bold */
    background: rgba(255, 255, 255,0.1); /* Add a blurred background */
    padding: 20px; /* Add some padding for better readability */
    border-radius: 10px; /* Add border radius for rounded corners */
    color:white;
    backdrop-filter: blur(10px); /* Apply a blur effect to the background */
}

        p {
            margin: 4px 0;
        }
.clock {
    position: absolute;
    top: 30px;
    left: 10px;
    width: 80px; /* Reduce width for a smaller size */
    height: 80px; /* Reduce height for a smaller size */
    border-radius: 50%; /* Make it circular */
    padding: 10px; /* Adjust padding for better spacing */
    font-size: 18px; /* Reduce font size for smaller text */
    color:white ; /* Set text color to blue */
    box-shadow: 0 2px 5px rgba(0, 0, 0, 0.3);
    text-align: center; /* Center text horizontally */
    line-height: 80px; /* Vertically center text */
    background: linear-gradient(45deg, #add8e6, #a3c2c2);
    animation: pulse 1s infinite alternate; /* Change animation to pulse every second */
}


@keyframes pulse {
    from {
        transform: scale(1);
    }
    to {
        transform: scale(1.1);
    }
}

    </style>
    <script>
        function startTime() {
            const today = new Date();
            let h = today.getHours();
            let m = today.getMinutes();
            let s = today.getSeconds();
            m = checkTime(m);
            s = checkTime(s);
            document.getElementById('clock').innerHTML =  h + ":" + m + ":" + s;
            setTimeout(startTime, 1000);
        }

        function checkTime(i) {
            if (i < 10) {i = "0" + i};  // add zero in front of numbers < 10
            return i;
        }
    </script>
</head>
<body onload="startTime()">
    <div class="background"></div>
    <div class="clock" id="clock"></div>
    <div class="content">
<header>
    <h1>QUIXY SEARCH ENGINE</h1>
    <p><strong>Quick Searches, Endless Discoveries.</strong></p>
</header>

        <form action="/search" method="post">
            <input type="text" name="query" placeholder="Enter search terms..." value="{{ query }}">
            <input type="submit" value="Search">
        </form>
        <div class="statistics">
            <p>Precision: {{ precision|default('N/A') }}</p>
            <p>Recall: {{ recall|default('N/A') }}</p>
            <p>F1 Score: {{ f1_score|default('N/A') }}</p>
            <p>About {{ result_count|default(0) }} results ({{ retrieval_time|default('N/A') }})</p>
            <p>You may search for: {{ expanded_query|default('N/A') }}</p>
        </div>
        <ul>
            {% if results %}
                {% for doc, link, text in results %}
                    <li>
                        <a href="{{ link }}">{{ text }}</a>
                        <p>Document ID: {{ doc }}</p>
                    </li>
                {% endfor %}
            {% else %}
                <li>No results found.</li>
            {% endif %}
        </ul>
    </div>
</body>
</html>



'''



if __name__ == "__main__":
    app.run(port=5000)


22:13:51.972 [main] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (11-li) - further warnings are suppressed
22:13:53.637 [main] WARN org.terrier.structures.indexing.Indexer - Indexed 113 empty documents
NGROK Tunnel URL: NgrokTunnel: "https://f960-34-23-52-87.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [11/May/2024 22:14:09] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/May/2024 22:14:10] "GET /favicon.ico HTTP/1.1" 404 -


Query: ahmed zewail
Processed Query: ahmed zewail


<ipython-input-40-51693c312d36>:117: FutureWarning: .transform() should be passed a dataframe. Use .search() to execute a single query.
  final_results = bm25_retriever.transform(expanded_query)


Retrieval Time: 3.02 seconds
Result Count: 173
exopanded query is :  ahmed zewail hassan egyptian who cairo center


INFO:werkzeug:127.0.0.1 - - [11/May/2024 22:14:18] "POST /search HTTP/1.1" 200 -


Query: ai
Processed Query: ai


<ipython-input-40-51693c312d36>:117: FutureWarning: .transform() should be passed a dataframe. Use .search() to execute a single query.
  final_results = bm25_retriever.transform(expanded_query)


Retrieval Time: 3.99 seconds
Result Count: 125
exopanded query is : artificial intelligence AI three-toed sloth artificial insemination Bradypus tridactylus ai Army Intelligence ai intelligence neural machine generative technology computer


INFO:werkzeug:127.0.0.1 - - [11/May/2024 22:14:40] "POST /search HTTP/1.1" 200 -


In [98]:
''' work google interface'''

import time
from flask import Flask, request, render_template_string
import pandas as pd
import pyterrier as pt
from sentence_transformers import SentenceTransformer, util
from pyngrok import ngrok
import nltk
from nltk.corpus import wordnet as wn
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

indexer = pt.DFIndexer("./myFirstIndex2", overwrite=True)
index_ref = indexer.index(df["processed_text"], df["docno"], df["Link"], df["Text"])
index = pt.IndexFactory.of(index_ref)
def calculate_precision(retrieved, relevant):

    true_positives = set(retrieved).intersection(relevant)
    if len(retrieved) == 0:
        return 0
    return len(true_positives) / len(retrieved)

def calculate_recall(retrieved, relevant):

    true_positives = set(retrieved).intersection(relevant)
    if len(relevant) == 0:
        return 0
    return len(true_positives) / len(relevant)

def calculate_f1(precision, recall):

    if (precision + recall) == 0:
        return 0
    return 2 * (precision * recall) / (precision + recall)


judgment = {
    'ahmed zewail': list(df.iloc[0:801]['docno']),
    'dr zewail': list(df.iloc[0:801]['docno']),
    'ahmed': list(df.iloc[0:801]['docno']),
    'nobel': list(df.iloc[0:801]['docno']),
    'zewail city for science and technology': list(df.iloc[801:1000]['docno']),
    'zewail city': list(df.iloc[801:1000]['docno']),
    'world health organization': list(df.iloc[1000:1600]['docno']),
    'who': list(df.iloc[1000:1600]['docno']),
    'healthline': list(df.iloc[1000:1600]['docno']),
    'health': list(df.iloc[1000:1600]['docno']),
    'ai': list(df.iloc[1600:1715]['docno']),
    'AI': list(df.iloc[1600:1715]['docno']),
    'artificial intelligence': list(df.iloc[1600:1715]['docno']),

}
app = Flask(__name__)



def get_synonyms(word):
    synonyms = set()
    for syn in wn.synsets(word):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name().replace('_', ' '))
    return synonyms





# # def expand_query_with_word2vec(query, terms, word2vec_model_path, top_n=20):

# #     word2vec_model = Word2Vec.load(word2vec_model_path)


# #     query_embedding = word2vec_model.wv[query]


# #     term_embeddings = [word2vec_model.wv[term] for term in terms]


# #     cos_scores = [word2vec_model.most_similar([query_embedding], [term_embedding])[0][0] for term_embedding in term_embeddings]


# #     related_terms = [terms[i] for i in np.argsort(cos_scores)[::-1][:top_n]]

# #     return related_terms
def expand_query_with_embeddings(query, model):
    query_embedding = model.encode(query, convert_to_tensor=True)

    term_embeddings = model.encode(terms, convert_to_tensor=True)
    cos_scores = util.pytorch_cos_sim(query_embedding, term_embeddings)[0]
    related_terms = []
    sorted_indices = cos_scores.argsort(descending=True)
    for i in range(7):
        related_terms.append(terms[sorted_indices[i]])



@app.route("/", methods=["GET"])
def home():
    return render_template_string(HTML_TEMPLATE)

@app.route("/search", methods=["POST"])
def search():
    start_time = time.time()
    query = request.form['query']
    processed_query = preprocess(query)
    print("Query:", query)
    print("Processed Query:", processed_query)

    synonyms = get_synonyms(processed_query)
    expanded_query = " ".join(synonyms)
    additional_terms = expand_query_with_embeddings(processed_query, model)
    expanded_query += " " + " ".join(additional_terms)

    bm25_retriever = pt.BatchRetrieve(index, wmodel="BM25", num_results=500, metadata=["docno", "Text"])
    final_results = bm25_retriever.transform(expanded_query)

    retrieval_duration = time.time() - start_time
    retrieval_time = f"{retrieval_duration:.2f} seconds"
    result_count = len(final_results)
    retrieval_duration = time.time() - start_time
    retrieval_time = f"{retrieval_duration:.2f} seconds"


    print("Retrieval Time:", retrieval_time)
    print("Result Count:", result_count)
    print("exopanded query is :",expanded_query)

    if 'Text' in final_results.columns:
        if query in judgment:
            retrieved_docnos = set(final_results['docno'].tolist())

            relevant_docnos = set(judgment[query])
            precision = calculate_precision(set(retrieved_docnos), relevant_docnos)
            recall = calculate_recall(set(retrieved_docnos), relevant_docnos)
            f1_score = calculate_f1(precision, recall)
        else:



            precision = 0
            recall = 0
            f1_score = 0

    result_count = len(final_results)
    search_results = []
    for idx, row in final_results.iterrows():
        docno = row['docno']
        link = df[df['docno'] == docno]['Link'].values[0]
        text = df[df['docno'] == docno]['Text'].values[0]
        search_results.append((docno, link, text))

    return render_template_string(HTML_TEMPLATE, query=query, results=search_results, precision=precision, recall=recall, f1_score=f1_score, retrieval_time=retrieval_time, result_count=result_count)

ngrok.set_auth_token('2feTTlkc3MwE8uIJWWxU5M4WRVC_7FhdKJQXYdDY7akug4zJn')
public_url = ngrok.connect(5000)
print('NGROK Tunnel URL:', public_url)

HTML_TEMPLATE = '''
<!doctype html>
<html lang="en">
<head>
 <p id="quote">Developed By : Hossam Amir, Supervisor : Dr Mohamed Maher </p>
  <p2 id="quote2">    </p2>
  <meta charset="utf-8">
  <title>محرك بحث الفيومي</title>
  <div>
            <p>Precision: {{ precision|default('N/A') }}</p>
            <p>Recall: {{ recall|default('N/A') }}</p>
            <p>F1 Score: {{ f1_score|default('N/A') }}</p>
        </div>
  <style>
    body, html {
      font-family: Arial, sans-serif;
      margin: 0;
      padding: 0;
      background: #fff;
    }
    header {
      padding-top: 120px;
      padding-bottom: 20px;
    }
    form {
      display: flex;
      justify-content: center;
    }
    input[type="text"] {
      width: 50%;
      height: 28px;
      padding: 10px;
      margin-right: 10px;
      font-size: 16px;
      border: 1px solid #dfe1e5;
      border-radius: 24px;
    }
    input[type="submit"] {
      padding: 10px 15px;
      font-size: 14px;
      background: #f8f9fa;
      border: 1px solid #f8f9fa;
      border-radius: 24px;
      cursor: pointer;
    }
    ul {
      list-style: none;
      margin-top: 5px;
      padding: 0;
      text-align: left;
    }
    li {
      background-color: #fff;
      padding: 12px 20px;
      margin: 8px 20%;
      border: 1px solid #ddd;
      border-radius: 8px;
      box-shadow: 0px 2px 5px rgba(0,0,0,0.1);
      transition: background-color 0.3s ease-in-out;
    }
    li:hover {
      background-color: #f8f9fa;
    }
    a {
      color: #1a0dab;
      text-decoration: none;
    }
    a:hover {
      text-decoration: underline;
    }
    p {
      font-size: small;
      color: #666;
    }
  </style>
</head>
<body>
  <header>
    <div style="text-align: center;">
      <img src="https://www.google.com/images/branding/googlelogo/1x/googlelogo_color_272x92dp.png" alt="Google">
    </div>
    <form action="/search" method="post">
      <input type="text" name="query" placeholder="Enter search terms..." value="{{ query }}">
      <input type="submit" value="Google Search">
    </form>
  </header>
  <ul>
    {% if query %}
      <p style="text-align: center; color: #777;">About {{ result_count }} results ({{ retrieval_time }})</p>
      {% for doc, link, text in results %}
        <li>
          <a href="{{ link }}" style="font-size: 16px;">{{ text }}</a>
          <p>Document ID: {{ doc }}</p>
        </li>
      {% endfor %}
    {% endif %}
  </ul>
</body>
</html>
'''


if __name__ == "__main__":
    app.run(port=5000)


18:25:57.691 [main] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (11-li) - further warnings are suppressed
18:26:00.079 [main] WARN org.terrier.structures.indexing.Indexer - Indexed 113 empty documents
NGROK Tunnel URL: NgrokTunnel: "https://d113-34-73-35-136.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [11/May/2024 18:26:04] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/May/2024 18:26:04] "GET /favicon.ico HTTP/1.1" 404 -


Query: Ahmed Zewail
Processed Query: ahmed zewail


<ipython-input-98-15ad11ad5bdb>:113: FutureWarning: .transform() should be passed a dataframe. Use .search() to execute a single query.
  final_results = bm25_retriever.transform(expanded_query)
INFO:werkzeug:127.0.0.1 - - [11/May/2024 18:26:36] "POST /search HTTP/1.1" 200 -


Retrieval Time: 3.21 seconds
Result Count: 153
exopanded query is :  ahmed zewail hassan egyptian who
Query: ahmed zewail
Processed Query: ahmed zewail


<ipython-input-98-15ad11ad5bdb>:113: FutureWarning: .transform() should be passed a dataframe. Use .search() to execute a single query.
  final_results = bm25_retriever.transform(expanded_query)
INFO:werkzeug:127.0.0.1 - - [11/May/2024 18:26:55] "POST /search HTTP/1.1" 200 -


Retrieval Time: 4.58 seconds
Result Count: 153
exopanded query is :  ahmed zewail hassan egyptian who


In [39]:
'''work fancy inter face'''

import time
from flask import Flask, request, render_template_string
import pandas as pd
import pyterrier as pt
from sentence_transformers import SentenceTransformer, util
from pyngrok import ngrok
import nltk
from nltk.corpus import wordnet as wn
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

indexer = pt.DFIndexer("./myFirstIndex2", overwrite=True)
index_ref = indexer.index(df["processed_text"], df["docno"], df["Link"], df["Text"])
index = pt.IndexFactory.of(index_ref)
def calculate_precision(retrieved, relevant):

    true_positives = set(retrieved).intersection(relevant)
    if len(retrieved) == 0:
        return 0
    return len(true_positives) / len(retrieved)

def calculate_recall(retrieved, relevant):

    true_positives = set(retrieved).intersection(relevant)
    if len(relevant) == 0:
        return 0
    return len(true_positives) / len(relevant)

def calculate_f1(precision, recall):

    if (precision + recall) == 0:
        return 0
    return 2 * (precision * recall) / (precision + recall)


judgment = {
    'ahmed zewail': list(df.iloc[0:801]['docno']),
    'dr zewail': list(df.iloc[0:801]['docno']),
    'ahmed': list(df.iloc[0:801]['docno']),
    'nobel': list(df.iloc[0:801]['docno']),
    'zewail city for science and technology': list(df.iloc[801:1000]['docno']),
    'zewail city': list(df.iloc[801:1000]['docno']),
    'world health organization': list(df.iloc[1000:1600]['docno']),
    'who': list(df.iloc[1000:1600]['docno']),
    'healthline': list(df.iloc[1000:1600]['docno']),
    'health': list(df.iloc[1000:1600]['docno']),
    'ai': list(df.iloc[1600:1715]['docno']),
    'AI': list(df.iloc[1600:1715]['docno']),
    'artificial intelligence': list(df.iloc[1600:1715]['docno']),

}
app = Flask(__name__)



def get_synonyms(word):
    synonyms = set()
    for syn in wn.synsets(word):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name().replace('_', ' '))
    return synonyms





# # def expand_query_with_word2vec(query, terms, word2vec_model_path, top_n=20):

# #     word2vec_model = Word2Vec.load(word2vec_model_path)


# #     query_embedding = word2vec_model.wv[query]


# #     term_embeddings = [word2vec_model.wv[term] for term in terms]


# #     cos_scores = [word2vec_model.most_similar([query_embedding], [term_embedding])[0][0] for term_embedding in term_embeddings]


# #     related_terms = [terms[i] for i in np.argsort(cos_scores)[::-1][:top_n]]

# #     return related_terms
def expand_query_with_embeddings(query, model):
    query_embedding = model.encode(query, convert_to_tensor=True)

    term_embeddings = model.encode(terms, convert_to_tensor=True)
    cos_scores = util.pytorch_cos_sim(query_embedding, term_embeddings)[0]
    related_terms = [terms[i] for i in cos_scores.argsort(descending=True)[:5]]
    return related_terms


@app.route("/", methods=["GET"])
def home():
    return render_template_string(HTML_TEMPLATE)

@app.route("/search", methods=["POST"])
def search():
    start_time = time.time()
    query = request.form['query']
    query = query.lower()
    processed_query = preprocess(query)
    print("Query:", query)
    print("Processed Query:", processed_query)

    synonyms = get_synonyms(processed_query)
    expanded_query = " ".join(synonyms)
    additional_terms = expand_query_with_embeddings(processed_query, model)
    expanded_query += " " + " ".join(additional_terms)

    bm25_retriever = pt.BatchRetrieve(index, wmodel="BM25", num_results=500, metadata=["docno", "Text"])
    final_results = bm25_retriever.transform(expanded_query)

    retrieval_duration = time.time() - start_time
    retrieval_time = f"{retrieval_duration:.2f} seconds"
    result_count = len(final_results)
    retrieval_duration = time.time() - start_time
    retrieval_time = f"{retrieval_duration:.2f} seconds"


    print("Retrieval Time:", retrieval_time)
    print("Result Count:", result_count)
    print("exopanded query is :",expanded_query)

    if 'Text' in final_results.columns:
        if query in judgment:
            retrieved_docnos = set(final_results['docno'].tolist())

            relevant_docnos = set(judgment[query])
            precision = calculate_precision(set(retrieved_docnos), relevant_docnos)
            recall = calculate_recall(set(retrieved_docnos), relevant_docnos)
            f1_score = calculate_f1(precision, recall)
        else:



            precision = 0
            recall = 0
            f1_score = 0

    result_count = len(final_results)
    search_results = []
    for idx, row in final_results.iterrows():
        docno = row['docno']
        link = df[df['docno'] == docno]['Link'].values[0]
        text = df[df['docno'] == docno]['Text'].values[0]
        search_results.append((docno, link, text))

    return render_template_string(HTML_TEMPLATE, query=query, results=search_results, precision=precision, recall=recall, f1_score=f1_score, retrieval_time=retrieval_time, result_count=result_count,expanded_query=expanded_query)

ngrok.set_auth_token('2feTTlkc3MwE8uIJWWxU5M4WRVC_7FhdKJQXYdDY7akug4zJn')
public_url = ngrok.connect(5000)
print('NGROK Tunnel URL:', public_url)

HTML_TEMPLATE = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Advanced Search Engine</title>
    <style>
        body, html {
            font-family: 'Arial', sans-serif;
            margin: 0;
            padding: 0;
            background: #f4f4f9;
            color: #333;
        }
        header {
            background: #0056b3;
            color: #fff;
            padding: 20px 40px;
            text-align: center;
        }
        form {
            display: flex;
            justify-content: center;
            margin-top: 20px;
        }
        input[type="text"], input[type="submit"] {
            padding: 10px;
            margin: 5px;
            border-radius: 5px;
            border: none;
        }
        input[type="text"] {
            flex: 1;
            max-width: 600px;
        }
        input[type="submit"] {
            background-color: #ff9900;
            color: white;
            cursor: pointer;
        }
        input[type="submit"]:hover {
            background-color: #e68a00;
        }
        ul {
            list-style: none;
            margin-top: 20px;
            padding: 0;
        }
        li {
            background-color: #ffffff;
            padding: 15px;
            margin: 10px 20%;
            border-radius: 8px;
            box-shadow: 0 2px 5px rgba(0,0,0,0.1);
        }
        li:hover {
            background-color: #f8f9fa;
        }
        a {
            color: #0066cc;
            text-decoration: none;
        }
        a:hover {
            text-decoration: underline;
        }
        p {
            margin: 5px 0;
        }
        .statistics {
            text-align: center;
            background-color: #eee;
            padding: 10px;
            border-radius: 5px;
            margin: 20px;
            font-size: 16px;
        }
    </style>
</head>
<body>
    <header>
        <h1>Advanced Search Engine</h1>
    </header>
    <form action="/search" method="post">
        <input type="text" name="query" placeholder="Enter search terms..." value="{{ query }}">
        <input type="submit" value="Search">
    </form>
    <div class="statistics">
        <p>Precision: {{ precision|default('N/A') }}</p>
        <p>Recall: {{ recall|default('N/A') }}</p>
        <p>F1 Score: {{ f1_score|default('N/A') }}</p>
        <p>About {{ result_count|default(0) }} results ({{ retrieval_time|default('N/A') }})</p>
        <p>You may search for:-> {{ expanded_query|default('N/A') }} </p>
    </div>
    <ul>
        {% if results %}
            {% for doc, link, text in results %}
                <li>
                    <a href="{{ link }}">{{ text }}</a>
                    <p>Document ID: {{ doc }}</p>
                </li>
            {% endfor %}
        {% else %}
            <li>No results found.</li>
        {% endif %}
    </ul>
</body>
</html>
'''




if __name__ == "__main__":
    app.run(port=5000)


22:12:07.937 [main] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (11-li) - further warnings are suppressed
22:12:09.471 [main] WARN org.terrier.structures.indexing.Indexer - Indexed 113 empty documents
NGROK Tunnel URL: NgrokTunnel: "https://01ce-34-23-52-87.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


In [ ]:


# import time
# from flask import Flask, request, render_template_string
# import pandas as pd
# import pyterrier as pt
# from sentence_transformers import SentenceTransformer, util
# from pyngrok import ngrok
# import nltk
# from nltk.corpus import wordnet as wn
# from nltk.stem import PorterStemmer
# from nltk.tokenize import word_tokenize

# indexer = pt.DFIndexer("./myFirstIndex2", overwrite=True)
# index_ref = indexer.index(df["processed_text"], df["docno"], df["Link"], df["Text"])
# index = pt.IndexFactory.of(index_ref)

# app = Flask(__name__)



# def get_synonyms(word):
#     synonyms = set()
#     for syn in wn.synsets(word):
#         for lemma in syn.lemmas():
#             synonyms.add(lemma.name().replace('_', ' '))
#     return synonyms





# # def expand_query_with_word2vec(query, terms, word2vec_model_path, top_n=20):

# #     word2vec_model = Word2Vec.load(word2vec_model_path)


# #     query_embedding = word2vec_model.wv[query]


# #     term_embeddings = [word2vec_model.wv[term] for term in terms]


# #     cos_scores = [word2vec_model.most_similar([query_embedding], [term_embedding])[0][0] for term_embedding in term_embeddings]


# #     related_terms = [terms[i] for i in np.argsort(cos_scores)[::-1][:top_n]]

# #     return related_terms
# def expand_query_with_embeddings(query, model):
#     query_embedding = model.encode(query, convert_to_tensor=True)

#     term_embeddings = model.encode(terms, convert_to_tensor=True)
#     cos_scores = util.pytorch_cos_sim(query_embedding, term_embeddings)[0]
#     related_terms = [terms[i] for i in cos_scores.argsort(descending=True)[:5]]
#     return related_terms


# @app.route("/", methods=["GET"])
# def home():
#     return render_template_string(HTML_TEMPLATE)

# @app.route("/search", methods=["POST"])
# def search():
#     start_time = time.time()
#     query = request.form['query']
#     processed_query = preprocess(query)
#     print("Query:", query)
#     print("Processed Query:", processed_query)

#     synonyms = get_synonyms(processed_query)
#     expanded_query = " ".join(synonyms)
#     additional_terms = expand_query_with_embeddings(processed_query, model)
#     expanded_query += " " + " ".join(additional_terms)

#     bm25_retriever = pt.BatchRetrieve(index, wmodel="BM25", num_results=30)
#     final_results = bm25_retriever.transform(expanded_query)

#     retrieval_duration = time.time() - start_time
#     retrieval_time = f"{retrieval_duration:.2f} seconds"
#     result_count = len(final_results)
#     retrieval_duration = time.time() - start_time
#     retrieval_time = f"{retrieval_duration:.2f} seconds"


#     print("Retrieval Time:", retrieval_time)
#     print("Result Count:", result_count)
#     print("exopanded query is :",expanded_query)




#     result_count = len(final_results)
#     search_results = []
#     for idx, row in final_results.iterrows():
#         docno = row['docno']
#         link = df[df['docno'] == docno]['Link'].values[0]
#         text = df[df['docno'] == docno]['Text'].values[0]
#         search_results.append((docno, link, text))

#     return render_template_string(HTML_TEMPLATE, query=query, results=search_results, retrieval_time=retrieval_time, result_count=result_count)

# ngrok.set_auth_token('2feTTlkc3MwE8uIJWWxU5M4WRVC_7FhdKJQXYdDY7akug4zJn')
# public_url = ngrok.connect(5000)
# print('NGROK Tunnel URL:', public_url)

# HTML_TEMPLATE = '''
# <!doctype html>
# <html lang="en">
# <head>
#  <p id="quote">Developed By : Hossam Amir, Supervisor : Dr Mohamed Maher </p>
#   <p2 id="quote2">    محركات البحث فن مش عن عن </p2>
#   <meta charset="utf-8">
#   <title>محرك بحث الفيومي </title>
#   <style>
#     body, html {
#       font-family: Arial, sans-serif;
#       margin: 0;
#       padding: 0;
#       background: #fff;
#     }
#     header {
#       padding-top: 120px;
#       padding-bottom: 20px;
#     }
#     form {
#       display: flex;
#       justify-content: center;
#     }
#     input[type="text"] {
#       width: 50%;
#       height: 28px;
#       padding: 10px;
#       margin-right: 10px;
#       font-size: 16px;
#       border: 1px solid #dfe1e5;
#       border-radius: 24px;
#     }
#     input[type="submit"] {
#       padding: 10px 15px;
#       font-size: 14px;
#       background: #f8f9fa;
#       border: 1px solid #f8f9fa;
#       border-radius: 24px;
#       cursor: pointer;
#     }
#     ul {
#       list-style: none;
#       margin-top: 5px;
#       padding: 0;
#       text-align: left;
#     }
#     li {
#       background-color: #fff;
#       padding: 12px 20px;
#       margin: 8px 20%;
#       border: 1px solid #ddd;
#       border-radius: 8px;
#       box-shadow: 0px 2px 5px rgba(0,0,0,0.1);
#       transition: background-color 0.3s ease-in-out;
#     }
#     li:hover {
#       background-color: #f8f9fa;
#     }
#     a {
#       color: #1a0dab;
#       text-decoration: none;
#     }
#     a:hover {
#       text-decoration: underline;
#     }
#     p {
#       font-size: small;
#       color: #666;
#     }
#   </style>
# </head>
# <body>
#   <header>
#     <div style="text-align: center;">
#       <img src="https://www.google.com/images/branding/googlelogo/1x/googlelogo_color_272x92dp.png" alt="Google">
#     </div>
#     <form action="/search" method="post">
#       <input type="text" name="query" placeholder="Enter search terms..." value="{{ query }}">
#       <input type="submit" value="Google Search">
#     </form>
#   </header>
#   <ul>
#     {% if query %}
#       <p style="text-align: center; color: #777;">About {{ result_count }} results ({{ retrieval_time }})</p>
#       {% for doc, link, text in results %}
#         <li>
#           <a href="{{ link }}" style="font-size: 16px;">{{ text }}</a>
#           <p>Document ID: {{ doc }}</p>
#         </li>
#       {% endfor %}
#     {% endif %}
#   </ul>
# </body>
# </html>
# '''


# if __name__ == "__main__":
#     app.run(port=5000)


In [55]:
def calculate_precision(retrieved, relevant):
    """Calculate the precision of the retrieved documents."""
    true_positives = set(retrieved).intersection(relevant)
    if len(retrieved) == 0:
        return 0
    return len(true_positives) / len(retrieved)

def calculate_recall(retrieved, relevant):
    """Calculate the recall of the retrieved documents."""
    true_positives = set(retrieved).intersection(relevant)
    if len(relevant) == 0:
        return 0
    return len(true_positives) / len(relevant)

def calculate_f1(precision, recall):
    """Calculate the F1 score from precision and recall."""
    if (precision + recall) == 0:
        return 0
    return 2 * (precision * recall) / (precision + recall)


judgment = {
    'ahmed zewail': list(df.iloc[0:801]['docno']),
    'dr zewail': list(df.iloc[0:801]['docno']),
    'ahmed': list(df.iloc[0:801]['docno']),
    'nobel': list(df.iloc[0:801]['docno']),
    'zewail city for science and technology': list(df.iloc[801:1000]['docno']),
    'zewail city': list(df.iloc[801:1000]['docno']),
    'world health organization': list(df.iloc[1000:1600]['docno']),
    'who': list(df.iloc[1000:1600]['docno']),
    'healthline': list(df.iloc[1000:1600]['docno']),
    'health': list(df.iloc[1000:1600]['docno']),
    'ai': list(df.iloc[1600:1715]['docno']),
    'AI': list(df.iloc[1600:1715]['docno']),
    'artificial intelligence': list(df.iloc[1600:1715]['docno']),

}


indexer = pt.DFIndexer("./myFirstIndex2", overwrite=True)
index_ref = indexer.index(df["processed_text"], df["docno"], df["Link"], df["Text"])
index = pt.IndexFactory.of(index_ref)


bm25_retriever = pt.BatchRetrieve(index, wmodel="BM25", num_results=1000, metadata=["docno"])


query = "zewail city for science and technology"
final_results = bm25_retriever.search(query)


retrieved_docnos = set(final_results['docno'].tolist())


relevant_docnos = set(judgment[query])


precision = calculate_precision(retrieved_docnos, relevant_docnos)
recall = calculate_recall(retrieved_docnos, relevant_docnos)
f1_score = calculate_f1(precision, recall)


print("Retrieved Document IDs:", retrieved_docnos)
print("Predefined Relevant Document IDs:", relevant_docnos)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)

14:40:31.828 [main] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (11-li) - further warnings are suppressed
14:40:34.381 [main] WARN org.terrier.structures.indexing.Indexer - Indexed 113 empty documents
Retrieved Document IDs: {'162-p', '27-li', '77-p', '58-li', '243-li', '167-p', '251-li', '33-li', '50-p', '582-li', '249-li', '218-li', '137-h1', '202-li', '37-h1', '22-li', '20-li', '74-li', '199-li', '19-li', '240-li', '78-p', '50-li', '74-p', '80-li', '579-li', '151-p', '64-p', '111-li', '272-li', '194-li', '247-li', '441-li', '241-li', '86-li', '59-li', '235-li', '69-li', '75-p', '270-li', '246-li', '61-p', '581-li', '29-li', '578-li', '62-li', '187-p', '165-p', '59-p', '206-li', '209-li', '56-p', '67-h3', '82-p', '200-li', '30-li', '260-li', '34-p', '234-li', '255-li', '290-li', '563-li', '25-li', '149-p', '16-li', '15-li', '232-li', '128-li', '233-li', '28-p', '2-p', '583-li', '88-li', '262-li', '189-p', '264-li', '245-li', '592-li', '26-h3',

In [46]:
print("Retrieved Document IDs:", [doc['docno'] for doc in final_results.to_dict(orient='records')])
print("Predefined Relevant Document IDs:", relevant_docnos)


Retrieved Document IDs: ['54-h1', '37-h1', '538-li', '53-li', '55-li', '58-li', '64-h1', '290-li', '413-li', '441-li', '473-li', '36-li', '56-li', '33-li', '208-h3', '56-li', '58-li', '270-li', '60-li', '59-li', '61-li', '30-li', '246-li', '62-li', '247-li', '249-li', '206-li', '234-li', '240-li', '245-li']
Predefined Relevant Document IDs: {'477-li', '336-li', '486-li', '243-li', '43-li', '310-li', '251-li', '374-li', '145-li', '68-p', '401-li', '570-li', '37-h1', '313-li', '22-li', '20-li', '360-li', '407-li', '104-li', '78-p', '64-li', '171-li', '505-li', '74-p', '107-li', '23-li', '80-li', '465-li', '80-h3', '148-li', '247-li', '441-li', '353-li', '376-li', '69-li', '304-li', '81-li', '246-li', '385-li', '553-li', '118-li', '630-li', '608-li', '136-li', '420-li', '539-li', '578-li', '581-li', '596-li', '84-p', '69-p', '483-li', '266-li', '179-p', '556-li', '288-li', '364-li', '24-li', '298-li', '472-li', '255-li', '381-li', '30-li', '234-li', '326-li', '137-li', '509-li', '394-li',

In [18]:
    # start_time = time.time()
    # query = request.form['query']
    # processed_query = preprocess(query)  # Make sure this is defined to clean the query

    # # Query expansion logic here
    # synonyms = get_synonyms(processed_query)
    # expanded_query = " ".join(synonyms)
    # additional_terms = expand_query_with_embeddings(processed_query, model)
    # expanded_query += " " + " ".join(additional_terms)

    # bm25_retriever = pt.BatchRetrieve(index, wmodel="BM25", num_results=30)
    # final_results = bm25_retriever.transform(expanded_query)

    # retrieval_duration = time.time() - start_time
    # retrieval_time = f"{retrieval_duration:.2f} seconds"
    # result_count = len(final_results)

    # # Convert DataFrame to list of dicts
    # results = final_results.to_dict('records')

    # return render_template_string(HTML_TEMPLATE, query=query, results=results, retrieval_time=retrieval_time)

Failed to retrieve data from https://www.sciencedirect.com/science/article/pii/S2090123220300934: 403 Client Error: Forbidden for url: https://www.sciencedirect.com/science/article/pii/S2090123220300934
Failed to retrieve data from https://www.openai.com/research/: 403 Client Error: Forbidden for url: https://www.openai.com/research/
Topic: Dr. Ahmed Zewail, Rows: 0 to 800
Topic: Zewail City of Science and Innovation, Rows: 801 to 999
Topic: Health-related topics, Rows: 1000 to 1599
Topic: Artificial Intelligence, Rows: 1600 to 1714


In [84]:
tokens2 = df['processed_text'].apply(word_tokenize)
tokens2
for i in range(len(tokens2)):
    for token in tokens2[i]:
        try:
            print(f'The total count of "{token}" is:', index.getLexicon()[token].getDocumentFrequency())
        except KeyError:
            print(f'Token "{token}" not found in the index')

Streaming output truncated to the last 5000 lines.
Token "original" not found in the index
The total count of "site" is: 5
Token "[" not found in the index
The total count of "4" is: 10
Token "]" not found in the index
Token "on" not found in the index
Token "may" not found in the index
The total count of "11" is: 5
The total count of "2011" is: 23
The total count of "cabinet" is: 2
Token "ministers" not found in the index
Token "issued" not found in the index
Token "decree" not found in the index
The total count of "establish" is: 6
Token "national" not found in the index
The total count of "project" is: 10
Token "scientific" not found in the index
Token "renaissance" not found in the index
Token "named" not found in the index
The total count of "zewail" is: 119
Token "city" not found in the index
Token "science" not found in the index
Token "technology" not found in the index
Token "the" not found in the index
Token "city" not found in the index
Token "officially" not found in the in

In [85]:
lex = index.getLexicon()
direct_index = index.getDirectIndex()
meta = index.getMetaIndex()
inv = index.getInvertedIndex()

for i in range(len(tokens2)):
    for token in tokens2[i]:
        le = lex.getLexiconEntry(token)
        word = token

        if le:
            doc_freq = le.getDocumentFrequency()

            doc_ids = []
            for posting in inv.getPostings(le):
                doc_id = meta.getItem("docno", posting.getId())
                doc_ids.append(doc_id)

            print(f"THe word '{word}' appeared {doc_freq} times. Job IDs: {doc_ids}")
        else:
            print(f"Token '{token}' not found in the index")


Streaming output truncated to the last 5000 lines.
Token 'original' not found in the index
THe word 'site' appeared 5 times. Job IDs: ['626-li', '58-p', '103-li', '4-h2', '22-p']
Token '[' not found in the index
THe word '4' appeared 10 times. Job IDs: ['149-p', '158-p', '226-li', '233-li', '235-li', '282-li', '626-li', '58-p', '103-li', '179-li']
Token ']' not found in the index
Token 'on' not found in the index
Token 'may' not found in the index
THe word '11' appeared 5 times. Job IDs: ['156-p', '63-article', '79-p', '56-p', '58-p']
THe word '2011' appeared 23 times. Job IDs: ['49-article', '60-p', '142-li', '143-li', '163-p', '165-p', '167-p', '216-li', '235-li', '236-li', '243-li', '244-li', '256-li', '257-li', '301-li', '425-li', '478-li', '56-p', '58-p', '87-li', '139-li', '148-li', '35-li']
THe word 'cabinet' appeared 2 times. Job IDs: ['56-p', '58-p']
Token 'ministers' not found in the index
Token 'issued' not found in the index
Token 'decree' not found in the index
THe word 'e